# 1. login 한 후, 게시판 옆에 user명 나오게 하기


- login한 username를 session 변수에넣어서 관리하기



In [ ]:
# base.html
'''

My blog<br>

사용자: {{username}}


<!--  나중에 바뀌는 부분-> sub template를 통해서 rendering되는 부분 -->
{% block content %}
{% endblock %}


copy right.....<br>
서울 특별시
'''

In [ ]:
# views.py
'''

def list(request):
    data =Post.objects.all()
    username = request.session.get('username', '')
    context = {'data': data,'username': username}
    return render (request,'blog/list.html', context)


class LoginView(View):
    def get(self, request):
        return render(request, 'blog/login.html')

    def post(self, request):

        # user table의 password는 암호화되어 들어가있음.-> 간접적으로 비교 해야 함.
        username = request.POST.get('username')
        password = request.POST.get('password')

        # 장고에서 지원하는 함수
        user = authenticate(username =username, password = password)
        if user == None:
            return redirect('login')

        request.session['username'] = username

        return redirect('list')

'''

# 2. html 파일에서 경로 해결

> urls.py에서
- /붙이면 directory 의미
- /없으면 resorce를 의미
- name을 지정하기 

- {url 'url-name' param1 param2, param3 %} 형태

In [ ]:
'''
<!-- import, includ 개념 -->
{% extends 'blog/base.html' %}


<!-- base template안의 위치 지정 -->

{% block content %}

    sub-template<br>

    {% for d in data%}

    <!-- get 방식으로 보내려면: a href: /blog/id =... -->
    <!-- 동적 url을 가져오려면 -->
        <!-- # 절대경로로 쓰기 위해서
        # 이름이 detail인 urls.py의 절대 경로를 계산함.
        # pk 파라미터가 있기 때문에: 제일 적합한 방법  -->
        <a href="{% url 'detail' d.pk %}">{{d.title}}</a><br>

        <!-- 절대 경로로 쓰는 것은 ok -->
        <a href="/blog/{{d.pk}}/detail">{{d.title}}</a><br>

        <!-- 정적 방법: 파라미터 네임벨유를 통한 방법 -->
        <a href="/blog/detail?id={{d.pk}}">{{d.title}}</a><br>


        <!-- # view 기반으로 하면서 상대경로로 쓰면 안됨. -->
        <a href="{{d.pk}}/detail">{{d.title}}</a><br>
        <!-- {{d.text|linebreaks}}    # 목록만 보여주도록 하기 위해서 빼기-->
        <!-- linebreaks: enter -> <br>로 바꿔줌 -->
        <hr>
    {%endfor%}


    

{% endblock %}



'''

In [1]:
> view 클래스를 사용하기 전에는
- 상대경로를 사용할 수 있음.

> view 클래스를 사용한 이후에는 
- 경로를 name으로 사용하기 때문 상대경로로 쓰면 안됨.
- 1. a href="{% url 'detail' d.pk %}">{{d.title}}</a><br>
- 2. 절대 경로( 동적 url 생성 방식):  <a href="/blog/{{d.pk}}/detail">{{d.title}}</a><br>
- 3. get 방식: <a href="/blog/detail?id={{d.pk}}">{{d.title}}</a><br>

SyntaxError: invalid syntax (<ipython-input-1-a9dadf6f449f>, line 1)

### view 클래스를 쓰는 이유
- get, post 방식을 구조화시켜서 사용하기 위함
- 하나의 클래스에서 get, post 방식으로 모두 사용하도록 하기 위해서 사용
- 모든 함수를 view 클래스를 쓸 필요는 없음.

# 3. 글 올리기

In [ ]:
# edit.html
'''


<!-- <form action = '' method = post> -->
    <!-- 정확하게 경로를 지정하기 위해서 urls.py에서 지정한 name이름 쓰기 -->
<form action="{% url 'add' %}" method = post>
    {% csrf_token %}
   title <input input = text name = 'title' ><br>
   text <textarea rows =10 clos = 30 name = 'text'><br>

    <input type="submit" value = '글 올리기'>
</form>


'''

In [ ]:
# urls.py
'''
 path('add/', views.PostView.as_view(), name = 'add')

'''

In [ ]:
# views.py
'''
class PostView(View):
        
    def get(self, request):
        return render(request, 'blog/edit.html')
    def post(self, request):

        title = request.POST.get('title')
        text = request.POST.get('text')
        username = request.session['username']
        user = User.objects.get(username = username)

        Post.objects.create(title = title, text=text, author = user)

        return HttpResponse('post ok')
'''

# 4. form 객체 사용해서 post수정하기

- 처음부터 작성하는 것이 아니라
- 데이터를 다시 읽어와서 default값으로 넣어주고, 수정해야 함.

### form 객체 이용하기
- db에 있는 내용을 읽고 쓰는 방법
- html에서 만든 input tag, 등의 ui요소를 장고가 만들어줌.
- html에서 연동된 field를 만들지 않고 form 객체를 이용하기

In [ ]:
# urls.py
'''
    path('<int:pk>/edit/', views.PostEditView.as_view(), name = 'edit')

'''


## 4.1. data 보여주기

> form instance를 만들어서 보여주는데
- 내가 수정할 글을 form 객체에 넣어서 초기하해서 보여주는 개념


> 실행
- 127.0.0.1:8000/blog/1/edit/

In [ ]:
# views.py
'''
class PostForm(Form):
    title = CharField(label = '제목', max_length = 20)
    text = CharField(label = '내용', widget = Textarea) # widget: ui를 의미



# 특정 post를 수정해야 함 -> 파라미터를 받아야 함(pk= id)
class PostEditView(View):
    def get(self, request, pk):

        # # 데이터에 초기값 지정
        # post = Post.objects.get(pk = pk)
        # # 모든 변수를 모두 매핑하면 복잡해지므로
        # # Form data를 사용하기
        # form = PostForm()
        post = get_object_or_404(Post, pk = pk)
        form = PostForm(initial = {'title': post.title, 'text': post.text})
        return render(request, 'blog/edit.html', {'form': form ,'pk':pk})

    def post(self, request, pk):
        # create가 아님
        # 이미 만들어진 객체를 갖고오기
        # 만든 객체를 rendering

        form = PostForm(request.POST)
        post = get_object_or_404(Post, pk = pk)
        post.title = form['title'].value()
        post.text = form['text'].value()
        post.publish()

        return redirect('list')
'''

In [ ]:
#edit.html
'''

<form action="{% url 'edit' pk %}" method=post>
    {% csrf_token %}
 
        {{ form.as_p }}
 
    <input type="submit" value="작성" >
 
 </form>
 
'''

### from객체 장점

> requiered 속성
- 속성 중 required: 반드시 존재해야 하는 값
- html 속성이고, 이 값이 있어서 서버로 전송해주는데, 
- 이 값이 없으면 전송을 안해줌.
- html에서 하려면, 하나하나 입력해줘야 하는데 
- 이 기능을 form이 대신 해줌


> validators기능
- 입력값이 정상값인지 아닌지 확인
- 길이가 너무 짧게 입력했을 때
- errer가 나오고 서버로 전송이 안됨.

In [ ]:
'''
def validator(value):
    if len(value) < 5 : raise  ValidationError('길이가 너무 짧아요')

class PostForm(Form):
    title = CharField(label = '제목', max_length = 20, validators=[validator])
    text = CharField(label = '내용', widget = Textarea) # widget: ui를 의미

'''